In [ ]:
# import Pkg; Pkg.add("Statistics")

In [51]:
using JuMP
using HiGHS
using Statistics
using Plots

# Constants and data initialization
# System physical constraints (volume limits in water units)
const V_MIN  = [150.0, 140.0, 110.0]
const V_MAX  = [250.0, 180.0, 150.0]
const V_CRIT = [300.0, 230.0, 200.0]
const V_INIT = [200.0, 160.0, 190.0]

# Penalty cost for violating soft constraints
const PENALTY = 10000.0 

# Deterministic demand profile
DEMAND_DATA = [248.0, 103.0, 85.0, 75.0, 65.0, 88.0, 510.0]

# Stochastic prices: Dict(Day => [Scenario 1, Scenario 2])
PRICES = Dict(
    1 => [950.0, 1050.0], 2 => [850.0, 1000.0], 3 => [120.0, 150.0],
    4 => [4.0, 5.0],      5 => [2.0, 10.0],     6 => [250.5, 300.0],
    7 => [741.0, 790.0]
)

# Stochastic inflows: Dict(Day => [Scenario 1 Vector, Scenario 2 Vector])
INFLOWS = Dict()
raw_inflows = [
    ([30.0, 9.0], [20.0, 0.0], [10.0, 5.0]),  # Mon
    ([31.0, 7.0], [30.0, 1.0], [11.0, 3.0]),  # Tue
    ([38.0, 5.0], [14.0, 2.0], [10.0, 3.0]),  # Wed
    ([40.0, 3.0], [13.0, 2.0], [11.0, 2.0]),  # Thu
    ([40.0, 1.0], [11.0, 0.0], [20.0, 0.0]),  # Fri
    ([26.0, 1.0], [19.0, 1.0], [22.0, 1.0]),  # Sat
    ([25.0, 0.0], [27.0, 0.0], [32.0, 0.0])   # Sun
]

for (d, day_data) in enumerate(raw_inflows)
    INFLOWS[d] = [
        [day_data[1][1], day_data[2][1], day_data[3][1]], # Scenario 1
        [day_data[1][2], day_data[2][2], day_data[3][2]]  # Scenario 2
    ]
end

# Structure for Benders cuts
struct Cut
    intercept::Float64
    slopes::Vector{Float64}
end

# Stage optimization problem
function solve_stage(stage_idx, prev_v, cuts, inflow, price, demand)
    model = Model(HiGHS.Optimizer)
    set_silent(model)

    # State and control variables
    @variable(model, V_MIN[i] <= v[i=1:3] <= V_CRIT[i]) # State: End volume
    @variable(model, u[1:3] >= 0)     # Control: Turbined flow
    # @variable(model, s[1:3] >= 0)
    @variable(model, over[1:3] >= 0)  # Slack: Volume violation
    @variable(model, thermal >= 0)    # Control: Thermal generation
    @variable(model, alpha >= 0)      # Value function approximation (cost-to-go)

    # Reservoir capacity constraints (soft)
    for i in 1:3
        @constraint(model, v[i] <= V_MAX[i] + over[i])
    end

    # Updated flow conservation constraints using cascade topology
    # v_t = v_{t-1} - u_t + inflow + u_{upstream}
    @constraint(model, Bal1, v[1] == prev_v[1] - u[1] + inflow[1])
    @constraint(model, Bal2, v[2] == prev_v[2] - u[2] + inflow[2] + u[1])
    @constraint(model, Bal3, v[3] == prev_v[3] - u[3] + inflow[3] + u[2])

    # Load balance constraint
    @constraint(model, sum(u) + thermal == demand)

    # Cutting planes approximation of future cost
    # alpha >= theta_k + <pi_k, v>
    for cut in cuts
        @constraint(model, alpha >= cut.intercept + sum(cut.slopes[i] * v[i] for i in 1:3))
    end

    # Objective of minimizing immediate operational cost + expected future cost
    @objective(model, Min, (price * thermal) + (PENALTY * sum(over)) + alpha)

    optimize!(model)

    # Return solution and duals for backward pass
    return (
        obj = objective_value(model),
        next_v = value.(v),
        duals = [dual(Bal1), dual(Bal2), dual(Bal3)], # Marginal water values
        thermal_gen = value(thermal),
        overflow_pen = sum(value.(over)) * PENALTY
    )
end

solve_stage (generic function with 1 method)

In [54]:
# SDDP training

# Initialize empty cut repository
stage_cuts = Dict{Int, Vector{Cut}}()
for t in 1:7; stage_cuts[t] = []; end

MAX_ITER = 1000
lower_bound_history = Float64[]
sim_cost_history = Float64[] # History for Upper Bound statistics

println("Starting SDDP loop (forward and backward passes)")
println(" Iter | Lower Bound (LB)  | Real Cost (Sim)    | UB Mean +- 95% CI")

for k in 1:MAX_ITER
    # Forward pass
    current_v = V_INIT
    trajectory = Dict{Int, Vector{Float64}}()
    trajectory[0] = V_INIT
    
    first_stage_obj = 0.0
    current_sim_cost = 0.0
    
    realizations = [rand(1:2) for _ in 1:7]
    
    for t in 1:7
        scen = realizations[t]
        res = solve_stage(t, current_v, stage_cuts[t], INFLOWS[t][scen], PRICES[t][scen], DEMAND_DATA[t])
        
        if t == 1; first_stage_obj = res.obj; end
        
        # Accumulate realized cost for print
        current_sim_cost += (PRICES[t][scen] * res.thermal_gen) + res.overflow_pen
        
        current_v = res.next_v
        trajectory[t] = current_v
    end
    
    push!(lower_bound_history, first_stage_obj)
    
    # Statistical Calculations (Upper Bound)
    push!(sim_cost_history, current_sim_cost)
    
    ub_mean_str = "-"
    
    # Calculate stats only after collecting enough samples
    if k > 1
        n_samples = length(sim_cost_history)
        ub_mean = mean(sim_cost_history)
        ub_std = std(sim_cost_history)
        
        # 95% Confidence Interval half-width: 1.96 * sigma / sqrt(N)
        ci_half_width = 1.96 * (ub_std / sqrt(n_samples))
        
        ub_mean_str = "$(round(ub_mean, digits=0)) ± $(round(ci_half_width, digits=0))"
    end

    # Backward pass
    # Iterate backwards to compute supporting hyperplanes
    for t in reverse(2:7)
        trial_v = trajectory[t-1]
        avg_pi  = [0.0, 0.0, 0.0]
        avg_obj = 0.0
        
        # Solve subproblems for all realizations to compute expectation
        for s_idx in 1:2
            res = solve_stage(t, trial_v, stage_cuts[t], INFLOWS[t][s_idx], PRICES[t][s_idx], DEMAND_DATA[t])
            avg_pi  += 0.5 * res.duals # Expected duals
            avg_obj += 0.5 * res.obj   # Expected cost
        end
        
        # Construct cut: intercept = E[obj] - <E[pi], trial_v>
        intercept = avg_obj - sum(avg_pi[i] * trial_v[i] for i in 1:3)
        push!(stage_cuts[t-1], Cut(intercept, avg_pi))
    end
    
    # Print metrics (Iter, LB, Real Cost, UB)
    println(" $(lpad(k,4)) | $(lpad(round(first_stage_obj, digits=2), 17)) | $(lpad(round(current_sim_cost, digits=2), 18)) | $(lpad(ub_mean_str, 22))")
end

Starting SDDP loop (forward and backward passes)
 Iter | Lower Bound (LB)  | Real Cost (Sim)    | UB Mean +- 95% CI
    1 |               0.0 |            59280.0 |                      -
    2 |          75406.29 |           185530.0 |    122405.0 ± 123725.0
    3 |          88741.71 |            86650.0 |     110487.0 ± 75155.0
    4 |          13965.21 |           182230.0 |     128422.0 ± 63718.0
    5 |          88741.71 |           429735.0 |    188685.0 ± 128012.0
    6 |          88741.71 |            48679.0 |    165351.0 ± 114089.0
    7 |           24926.2 |           13777.45 |    143697.0 ± 105350.0
    8 |          70630.23 |            31498.0 |     129672.0 ± 95287.0
    9 |         104862.94 |           119779.0 |     128573.0 ± 84063.0
   10 |          70630.23 |            59099.0 |     121626.0 ± 76411.0
   11 |          70662.15 |            26150.0 |     112946.0 ± 71179.0
   12 |         104892.03 |           324485.0 |     130574.0 ± 73593.0
   13 |          810

In [ ]:
# # function to delete all learned cuts for greedy lakes (used only once)
# stage_cuts = Dict{Int, Vector{Cut}}()
# for t in 1:7
#     stage_cuts[t] = []
# end

# # clear history logs so plots start fresh
# lower_bound_history = Float64[]
# sim_cost_history = Float64[]

Float64[]

In [55]:
# Test Scenario

println("\nRunning validation scenario:")

test_prices = [950.0, 850.0, 120.0, 5.0, 2.0, 250.5, 741.0]
test_inflows = [
    [31.0, 21.0, 11.0], [30.0, 29.0, 10.0], [37.0, 13.0, 9.0],
    [13.0, 3.0, 3.0],   [39.0, 12.0, 21.0], [24.0, 18.0, 21.0],
    [27.0, 25.0, 30.0]
]

current_v = V_INIT
total_cash_cost = 0.0

println(" Day |  Cost (G) | Thermal |     L1 Level        |     L2 Level        |     L3 Level        | Status") # L - Lake

for t in 1:7
    # Solve stage with current state and test data
    res = solve_stage(t, current_v, stage_cuts[t], test_inflows[t], test_prices[t], DEMAND_DATA[t])
    
    # Calculate costs
    daily_bill = (test_prices[t] * res.thermal_gen) + res.overflow_pen
    total_cash_cost += daily_bill
    
    # Water flow analysis
    v_L1 = round(res.next_v[1], digits=1)
    v_L2 = round(res.next_v[2], digits=1)
    v_L3 = round(res.next_v[3], digits=1)
    
    # Logic for status string
    status_parts = []
    if res.thermal_gen > 0.1
        push!(status_parts, "HEDGE (Buy Power)")
    end
    if res.overflow_pen > 0
        push!(status_parts, "FLOOD (Damage)")
    end
    if isempty(status_parts)
        push!(status_parts, "Hydro Only")
    end
    status_str = join(status_parts, " + ")

    println("  $t  | $(lpad(round(daily_bill, digits=1), 9)) | $(lpad(round(res.thermal_gen, digits=1), 7)) | $(lpad(v_L1, 6)) / 250.0      | $(lpad(v_L2, 6)) / 180.0      | $(lpad(v_L3, 6)) / 150.0      | $status_str")
    
    # Update state for next day
    current_v = res.next_v
end

println("Final true cost: $(round(total_cash_cost, digits=2)) G")


Running validation scenario:
 Day |  Cost (G) | Thermal |     L1 Level        |     L2 Level        |     L3 Level        | Status
  1  |       0.0 |     0.0 |  206.0 / 250.0      |  140.0 / 180.0      |  110.0 / 150.0      | Hydro Only
  2  |       0.0 |     0.0 |  224.3 / 250.0      |  140.0 / 180.0      |  110.0 / 150.0      | Hydro Only
  3  |       0.0 |     0.0 |  218.0 / 250.0      |  160.0 / 180.0      |  150.0 / 150.0      | Hydro Only
  4  |      80.0 |    16.0 |  221.0 / 250.0      |  161.0 / 180.0      |  128.0 / 150.0      | HEDGE (Buy Power)
  5  |       0.0 |     0.0 |  247.3 / 250.0      |  179.0 / 180.0      |  110.0 / 150.0      | Hydro Only
  6  |    2254.5 |     9.0 |  250.0 / 250.0      |  180.0 / 180.0      |  150.0 / 150.0      | HEDGE (Buy Power)
  7  |       0.0 |     0.0 |  173.7 / 250.0      |  140.0 / 180.0      |  110.0 / 150.0      | Hydro Only
Final true cost: 2334.5 G


In [ ]:
# Visualization
default(size=(800, 600), margin=5Plots.mm)

# Create the directory if it doesn't exist
mkpath("figures")

println("\nGenerating visualizations...")

# Data collection (re-running validation to capture specific plotting vectors)
plot_days = 0:7
val_thermal = Float64[]
val_prices  = test_prices # From previous block
val_l1 = [V_INIT[1]]; val_l2 = [V_INIT[2]]; val_l3 = [V_INIT[3]]

sim_v = V_INIT
for t in 1:7
    res = solve_stage(t, sim_v, stage_cuts[t], test_inflows[t], test_prices[t], DEMAND_DATA[t])
    push!(val_thermal, res.thermal_gen)
    push!(val_l1, res.next_v[1])
    push!(val_l2, res.next_v[2])
    push!(val_l3, res.next_v[3])
    global sim_v = res.next_v
end


# Visualization 1: Plot to explain low costs
# proves the model only buys power when market prices crash
p1 = bar(1:7, val_thermal, label="Thermal Generation", 
         color=:red, alpha=0.5, ylabel="Energy Units", xlabel="Day",
         title="Smart Buying: Arbitrage Strategy", legend=:topleft)

plot!(twinx(), 1:7, val_prices, label="Market Price", 
      color=:green, linewidth=3, ylabel="Price (G)", legend=:topright)

savefig("figures/1_arbitrage_strategy.png")
println("Saved 'figures/1_arbitrage_strategy.png'")


# Visualization 2: The cascaded system of the 3 lakes
l1_plot = plot(plot_days, val_l1, title="Lake 1 (Top)", ylabel="Vol", label="", 
               color=:blue, fill=(0, 0.1, :blue), ylims=(150, 260))
l2_plot = plot(plot_days, val_l2, title="Lake 2 (Middle)", ylabel="Vol", label="", 
               color=:cyan, fill=(0, 0.1, :cyan), ylims=(140, 190))
l3_plot = plot(plot_days, val_l3, title="Lake 3 (Bottom)", ylabel="Vol", label="", 
               color=:teal, fill=(0, 0.1, :teal), ylims=(110, 160))

# Combine into one 3-panel figure
p2 = plot(l1_plot, l2_plot, l3_plot, layout=(3,1), size=(600, 800))

savefig("figures/2_cascade_levels.png")
println("Saved 'figures/2_cascade_levels.png'")

# Visualization 3: The Spaghetti Plot / Risk Analysis running 50 Monte Carlo simulations to show the "Cone of Uncertainty"
p3 = plot(title="Risk Analysis: Cone of Uncertainty (Lake 1)", 
          ylabel="Volume", xlabel="Day", legend=false, ylims=(140, 260))

# Reference constraints
hline!(p3, [250.0], color=:black, linestyle=:dot)
hline!(p3, [150.0], color=:red, linestyle=:dot)

# Run 50 random simulations using the trained policy
println("Running 50 simulations for Spaghetti Plot...")
for i in 1:50
    sim_traj = [V_INIT[1]] # Start at initial
    curr_sim_v = V_INIT
    sim_realizations = [rand(1:2) for _ in 1:7]
    
    for t in 1:7
        # Forward pass logic
        scen = sim_realizations[t]
        res = solve_stage(t, curr_sim_v, stage_cuts[t], INFLOWS[t][scen], PRICES[t][scen], DEMAND_DATA[t])
        curr_sim_v = res.next_v
        push!(sim_traj, curr_sim_v[1])
    end
    plot!(p3, plot_days, sim_traj, color=:blue, alpha=0.15)
end

# Overlay the specific Test Scenario in thick red
plot!(p3, plot_days, val_l1, color=:red, linewidth=3, label="Test Scenario")

savefig("figures/3_risk_spaghetti.png")
println("Saved 'figures/3_risk_spaghetti.png'")

# Visualization 4: Convergence Plot
# shows how the lower bound stabilized over iterations
# (uses the 'lower_bound_history' from the training block)
p4 = plot(1:length(lower_bound_history), lower_bound_history, 
     title="SDDP Convergence (Learning Curve)", 
     xlabel="Iteration", 
     ylabel="Lower Bound (Estimated Cost)",
     linewidth=2, color=:purple, legend=false)

savefig("figures/4_convergence.png")
println("Saved 'figures/4_convergence.png'")

# Visualization 5: Policy Comparison with all lakes
# Data from the Greedy/Myopic run (Cost=0)
greedy_l1 = [200.0, 206.0, 224.3, 244.7, 235.7, 250.0, 250.0, 165.3]
greedy_l2 = [160.0, 140.0, 140.0, 140.0, 140.0, 167.0, 180.0, 140.0]
greedy_l3 = [190.0, 110.0, 110.0, 110.0, 110.0, 110.0, 125.0, 110.0]

p5_1 = plot(0:7, val_l1, label="SDDP (L1)", color=:blue, marker=:circle, title="Lake 1", ylabel="Vol", legend=:bottomright)
plot!(p5_1, 0:7, greedy_l1, label="Greedy", color=:red, linestyle=:dash, marker=:x)
hline!(p5_1, [250.0, 150.0], color=:grey, linestyle=:dot, label="")

p5_2 = plot(0:7, val_l2, label="SDDP (L2)", color=:cyan, marker=:circle, title="Lake 2", ylabel="Vol", legend=:bottomright)
plot!(p5_2, 0:7, greedy_l2, label="Greedy", color=:red, linestyle=:dash, marker=:x)
hline!(p5_2, [180.0, 140.0], color=:grey, linestyle=:dot, label="")

p5_3 = plot(0:7, val_l3, label="SDDP (L3)", color=:teal, marker=:circle, title="Lake 3", ylabel="Vol", legend=:bottomright)
plot!(p5_3, 0:7, greedy_l3, label="Greedy", color=:red, linestyle=:dash, marker=:x)
hline!(p5_3, [150.0, 110.0], color=:grey, linestyle=:dot, label="")

p5 = plot(p5_1, p5_2, p5_3, layout=(3,1), size=(600, 800))

savefig("figures/5_policy_comparison.png")
println("Saved 'figures/5_policy_comparison.png'")

# Visualization 6: Convergence zoom with first 200 iterations
zoom_limit = min(200, length(lower_bound_history))
p6 = plot(1:zoom_limit, lower_bound_history[1:zoom_limit], 
     title="SDDP Convergence (First 200 Iterations)", 
     xlabel="Iteration", 
     ylabel="Lower Bound",
     linewidth=2, color=:purple, legend=false)

savefig("figures/6_convergence_zoom.png")
println("Saved 'figures/6_convergence_zoom.png'")

println("Done")


Generating visualizations...
Saved 'figures/1_arbitrage_strategy.png'
Saved 'figures/2_cascade_levels.png'
Running 50 simulations for Spaghetti Plot...
Saved 'figures/3_risk_spaghetti.png'
Saved 'figures/4_convergence.png'
Saved 'figures/5_policy_comparison.png'
Saved 'figures/6_convergence_zoom.png'
Done


The results demonstrate a transition from a "myopic" (greedy) approach to a risk-averse strategy.

Strategic Arbitrage and Economic Efficiency plots. The most critical insight is revealed in the "Smart Buying" visualization. The model demonstrates sophisticated economic reasoning by performing inter-temporal arbitrage. Rather than simply using water to avoid all costs, the model voluntarily purchases thermal generation on Day 4 and Day 6. This decision correlates perfectly with the market price crash (dropping to ~5 G) and the need to maximize storage before Sunday. The model effectively "buys" water storage by substituting it with cheap thermal power, preserving the reservoirs for the high-price, high-demand scenario on Sunday (Day 7).

Risk aversion vs. myopic behavior comparative analysis of Lake 1 levels highlights the cost of security. The "Greedy" policy (red line) drains the reservoir faster to minimize immediate costs, leaving the system vulnerable. In contrast, the SDDP policy (blue line) maintains a higher "head" or safety buffer. This is necessary because of the volatility shown in the "Cone of Uncertainty". With the spillage variable removed, the model must now balance two opposing risks: running dry and overflowing. The SDDP policy keeps the reservoir high enough to avoid shortage but low enough to absorb unexpected inflows, surviving all potential futures.

The cascaded nature of the river system is managed effectively, as evidenced by the multi-lake plots. A key observation is the immediate correction of Lake 3, which initializes at 190. The model instantly drains this excess volume on Day 1 to avoid penalties, using Lake 3 as a high-throughput "swing" resource that oscillates rapidly between its minimum at 110 and maximum at 150 to absorb flows. Meanwhile, Lake 1 (the top reservoir) is treated as the primary stable battery; the model steadily builds its volume from 206 up to the 250 maximum by Day 6. This synchronized operation prevents wastage in the smaller, volatile downstream lakes while securing the bulk of the system's potential energy in the massive upstream reservoir.

Removing the free spillage variable led the optimization to follow a narrower operating range, keeping reservoir levels high enough to avoid shortages but safely below the maximum to prevent the penalty. As a result, the training log increased in variability. This indicates that the final policy balancing energy objectives with the need to stay within physical infrastructure limits under uncertain weather conditions.

The learning curve provides definitive proof that the SDDP algorithm successfully converged on an optimal policy. The Lower Bound rose rapidly in the first 50 iterations and then stabilized, indicating that the model quickly grasped the future value of water and the penalties for shortage. Furthermore, the extended 1000-iteration training log validates this convergence statistically. The Statistical Upper Bound (the rolling mean of real costs) settled decisively around 125000 G, with a narrowing 95% confidence interval of approximately 6,000 G. This statistical stability confirms that the policy has found an equilibrium, balancing the high immediate cost of thermal generation against the long-term stochastic risk of future shortages and flood penalties.

In conclusion, the SDDP model establishes that paying a small "insurance premium" during cheap and volatile market periods is mathematically superior to the alternative, proving that optimal reservoir management requires a defense against both catastrophic energy shortages and costly infrastructure flood damages.